## MIE424 (2021 Winter) Lab 2

Yuehuan He and Jake Mosseri 

In this lab, you will be using linear classifier to build a spam classifier. 

Many email services today provide spam filters that are able to classify emails into spam and non-spam email with high accuracy. You will use linear classifier to build your own spam filter.

You will be training a classifier to classify whether a given email, $x$, is spam ($y = 1$) or non-spam ($y = 0$). In particular, you need to convert each email into a feature vector $x \in R^n$. The following parts of the lab will walk you through how such a feature vector can be constructed from an email.

Dataset used:

- spamTrain.mat - Spam training set
- spamTest.mat - Spam test set



The dataset has been processed using the following feature extraction. n = # words in vocabulary list. Specifically, the feature $x_i \in \{0, 1\}$ for an email corresponds to whether the $i$-th word in the dictionary occurs in the email. That is, $x_i = 1$ if the $i$-th word is in the email and $x_i$ = 0 if the $i$-th word is not present in the email.



Preprocessing process:

-  Lower-casing: The entire email is converted into lower case, so that captialization is ignored (e.g., IndIcaTE is treated the same as Indicate).
- Stripping HTML: All HTML tags are removed from the emails. Many emails often come with HTML formatting; we remove all the HTML tags, so that only the content remains.
- Normalizing URLs: All URLs are replaced with the text “httpaddr”.
- Normalizing Email Addresses: All email addresses are replaced with the text “emailaddr”.
- Normalizing Numbers: All numbers are replaced with the text “number”.
- Normalizing Dollars: All dollar signs are replaced with the text “dollar”.
- Word Stemming: Words are reduced to their stemmed form. For example, “discount”, “discounts”, “discounted” and “discounting” are all replaced with “discount”. Sometimes, the Stemmer actually strips off additional characters from the end, so “include”, “includes”, “included”, and “including” are all replaced with “includ”.
- Removal of non-words: Non-words and punctuation have been removed. All white spaces (tabs, newlines, spaces) have all been trimmed to a single space character.

After preprocessing the emails, we have a list of words for each email. The next step is to choose which words we would like to use in our classifier and which we would want to leave out.

For this example, we have chosen only the most frequently occuring words as our set of words considered (the vocabulary list). Since words that occur rarely in the training set are only in a few emails, they might cause the model to overfit our training set. The complete vocabulary list is in the file vocab.txt and also shown in the figure below. Our vocabulary list was selected by choosing all words which occur at least a 100 times in the spam corpus, resulting in a list of 1899 words. In practice, a vocabulary list with about 10,000 to 50,000 words is often used.


![Encoding process](img/encoding.png)


Given the vocabulary list, we can now map each word in the preprocessed emails into a list of word indices that contains the index of the word in the vocabulary list. The following figure shows the mapping for the sample email. Specifically, in the sample email, the word “anyone” was first normalized to “anyon” and then mapped onto the index 86 in the vocabulary list.




![Encoded datapoint](img/encodedx.png)

### Load Necessary Libraries

In [1]:
try:
    import nltk
except ImportError:
    !pip install nltk
    import nltk
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from scipy.io import loadmat

### Import Dataset

Now we load a preprocessed training dataset that will be used to train a linear classifier. spamTrain.mat contains training examples of spam and non-spam email, while spamTest.mat contains test examples. Each original email was processed and converted into a vector $x_{(i)} \in R^{1899}$.


In [2]:
# Load the Spam Email dataset
train_data = loadmat('spamTrain.mat')
test_data = loadmat('spamTest.mat')
X_train = train_data['X']
Y_train = train_data['y'].ravel()
X_test  = test_data['Xtest']
Y_test  = test_data['ytest'].ravel()

In [3]:
# Load vocabulary list:
vocabulary = []
with open('vocab.txt') as f:
    for line in f:
        idx, word = line.split('\t')
        vocabulary.append(word.strip())

Consider the following example email: 

*Anyone knows how much it costs to host a web portal ?*
*Well, it depends on how many visitors youre expecting. This can be anywhere from less than 10 bucks a month to a couple of 100. You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 if youre running something big..*
*To unsubscribe yourself from this mailing list, send an email to:
groupname-unsubscribe@egroups.com*

The encoded representation of this email is given in the first figure:

[86, 916, 794, 1077, ...]

To check this:

In [4]:
vocabulary[85]

'anyon'

In [5]:
vocabulary[915]

'know'

Recap from Lab 1:

- How to find the size of training set and test set?

In [6]:
print(f'There are {X_train.shape[0]} training examples and {X_test.shape[0]} test examples')

There are 4000 training examples and 1000 test examples


### Construct and Train a Linear Classifier

After loading the dataset, we then proceed to train a linear classifier with hinge loss to classify between spam ($y = 1$) and non-spam ($y = 0$) emails.

Here we use the `SGDClassifier` in sklearn package to construct and train a linear classifier:

class sklearn.linear_model.SGDClassifier(loss='hinge', *, penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)

Parameters:

- **loss**: str, default=’hinge’
    - The loss function to be used. Defaults to ‘hinge’.
    - The possible options are ‘hinge’, ‘squared_hinge’, etc.
- **penalty**: {‘l2’, ‘l1’, ‘elasticnet’}, default=’l2’
    - The penalty (aka regularization term) to be used. Defaults to ‘l2’ which is the standard regularizer for linear SVM models. ‘l1’ and ‘elasticnet’ might bring sparsity to the model (feature selection) not achievable with ‘l2’.
- **alpha**: float, default=0.0001
    - Constant that multiplies the regularization term. The higher the value, the stronger the regularization. 


See more in its [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)



In [7]:
clf = make_pipeline(StandardScaler(),SGDClassifier(alpha=0,learning_rate='invscaling',eta0=0.01,verbose=1))

# Here alpha = 0 is used to not apply any penalty. Learning rate then cannot take the default value 'optimal'
# and an initial stepsize needs to be specified.

In [8]:
clf.fit(X_train, Y_train)

-- Epoch 1
Norm: 0.98, NNZs: 1895, Bias: -0.206604, T: 4000, Avg. loss: 0.166303
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 1.00, NNZs: 1895, Bias: -0.233591, T: 8000, Avg. loss: 0.073208
Total training time: 0.03 seconds.
-- Epoch 3
Norm: 1.01, NNZs: 1895, Bias: -0.247319, T: 12000, Avg. loss: 0.057050
Total training time: 0.04 seconds.
-- Epoch 4
Norm: 1.03, NNZs: 1895, Bias: -0.256208, T: 16000, Avg. loss: 0.047773
Total training time: 0.05 seconds.
-- Epoch 5
Norm: 1.04, NNZs: 1895, Bias: -0.263470, T: 20000, Avg. loss: 0.042022
Total training time: 0.07 seconds.
-- Epoch 6
Norm: 1.04, NNZs: 1895, Bias: -0.268784, T: 24000, Avg. loss: 0.037829
Total training time: 0.08 seconds.
-- Epoch 7
Norm: 1.05, NNZs: 1895, Bias: -0.273459, T: 28000, Avg. loss: 0.034526
Total training time: 0.10 seconds.
-- Epoch 8
Norm: 1.06, NNZs: 1895, Bias: -0.276592, T: 32000, Avg. loss: 0.031960
Total training time: 0.11 seconds.
-- Epoch 9
Norm: 1.07, NNZs: 1895, Bias: -0.280032, T: 36000, Avg.

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('sgdclassifier',
                 SGDClassifier(alpha=0, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.01,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='invscaling', loss='hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None,
                               penalty='l2', power_t=0.5, random_state=None,
                               shuffle=True, tol=0.001, validation_fraction=0.1,
                               verbose=1, warm_start=False))],
         verbose=False)

### Evaluate the fitted model

In [9]:
# Training accuracy
prediction_train = clf.predict(X_train)
print ('Train Accuracy:', np.mean(prediction_train == Y_train) * 100,'%')

Train Accuracy: 99.35000000000001 %


In [10]:
# Test accuracy
prediction_test = clf.predict(X_test)
print ('Test Accuracy:', np.mean(prediction_test == Y_test) * 100,'%')

Test Accuracy: 97.8 %


### Discussion

To better understand how the spam classifier works, we can inspect the parameters to see which words the classifier thinks are the most predictive of spam. The next step of this lab finds the parameters with the largest positive values in the classifier and displays the corresponding words. Thus, if an email contains words such as “click”, “remove”, “gurantee” etc., it is likely to be classified as spam.

In [11]:
coef = clf.named_steps['sgdclassifier'].coef_.ravel()
idx  = coef.argsort()[::-1].ravel()
print ('Top predictors of spam:')
for i in range(15):
    print (vocabulary[idx[i]], coef[idx[i]])

Top predictors of spam:
click 0.1794555324960575
remov 0.15772664712361162
our 0.12176842175777475
telephon 0.11815650661818612
here 0.11501816402040703
below 0.11276389032880309
nbsp 0.10881947020000925
guarante 0.10551974303122025
price 0.09104807991520399
pleas 0.08813961465097749
estat 0.08647498294887866
fill 0.0856762101796141
basenumb 0.08271818938311372
absolut 0.07893865901230397
receiv 0.07880284147332711


Exercise:
- try train the model with other loss functions;
- find an email of your own and see if the fitted model can classify it correctly (the following helper functions might be useful in preprocessing your email content)
- train the model with a subset of the training data and see how the test accuracies compare, e.g. with 100% of training data vs with 50%.

In [12]:
import re
from nltk import PorterStemmer

def split(delimiters, string, maxsplit=0):
    pattern = '|'.join(map(re.escape, delimiters))
    return re.split(pattern, string, maxsplit)

def process_email(email_contents,vocabulary):
    """
    Preprocesses a the body of an email and returns a list of word indices.
    Parameters
    ----------
    email_contents : string
        The email content.
    Returns
    -------
    list
        A list of word indices.
    """
    email_contents = email_contents.lower()
    email_contents = re.sub('<[^<>]+>', ' ', email_contents)
    email_contents = re.sub('[0-9]+', 'number', email_contents)
    email_contents = re.sub('(http|https)://[^\s]*', 'httpaddr', email_contents)
    email_contents = re.sub('[^\s]+@[^\s]+', 'emailaddr', email_contents)
    email_contents = re.sub('[$]+', 'dollar', email_contents)

    words = split(""" @$/#.-:&*+=[]?!(){},'">_<;%\n\r""", email_contents)
    word_indices = []
    stemmer = PorterStemmer()
    for word in words:
        word = re.sub('[^a-zA-Z0-9]', '', word)
        if word == '':
            continue
        word = stemmer.stem(word)
        if word in vocabulary:
            idx = vocabulary.index(word)
            word_indices.append(idx)

    return word_indices

Exercise:

- try train the model with other loss functions;
    - to apply different loss function, you could change the parameter in `SGDClassifier`.
    - for example: 'perceptron' loss (which we would be implementing from sketch in next Lab): SGDClassifier(loss = 'perceptron', alpha=0, learning_rate='invscaling', eta0=0.01, verbose=1)
    - The loss function to be used. Defaults to ‘hinge’, which gives a linear SVM. The possible options are ‘hinge’, ‘log’, ‘modified_huber’, ‘squared_hinge’, ‘perceptron’, or a regression loss: ‘squared_loss’, ‘huber’, ‘epsilon_insensitive’, or ‘squared_epsilon_insensitive’.
    - The ‘log’ loss gives logistic regression, a probabilistic classifier. ‘modified_huber’ is another smooth loss that brings tolerance to outliers as well as probability estimates. ‘squared_hinge’ is like hinge but is quadratically penalized. ‘perceptron’ is the linear loss used by the perceptron algorithm. The other losses are designed for regression but can be useful in classification as well; see SGDRegressor for a description.

- find an email of your own and see if the fitted model can classify it correctly (the following helper functions might be useful in preprocessing your email content)

In [14]:
email_example = 'Heres what you need to know. audible   WISH LIST   BEST SELLERS   NEW RELEASES Its great to be a member!  Just listen to these perks Get One Audiobook Every Month Members get 1 credit a month — good for any title regardless of length or price. Explore our catalogue Receive a BONUS audiobook each month. Editors Extra gives you a free monthly audiobook chosen by our editors as part of your membership. Add to Your Library Exclusive Member Discounts Never pay full price!  Used up your monthly credits? Members can buy additional titles at any time for 30% off the list price, plus get access to exclusive sales and daily deals. Shop Now Audible Original Podcasts Unlimited listening to gripping and immersive documentaries, comedy, and more. 100% free for members. Download to lots of devices Your library is automatically synced across all your devices, so you can listen anytime, anywhere. See listening options Need help? From app questions to listening recommendations, our Customer Care team has got you covered. Contact Customer Care      Give them gifts for life   audible gift centre    Follow us on social   facebook   instagram   twitter         Unsubscribe | Help Centre | Account Details | Conditions of Use | Privacy Policy  Contact Customer Service for any questions, or if you need assistance.  You may adjust your communication preferences by updating your notification settings.  ©1997-2021 Audible, Inc. Audible and the Audible logo are trademarks of Audible, Inc., or its affiliates. 1 Washington Park, 16th Floor, Newark, NJ 07102. All Rights Reserved. '
print(email_example)

Heres what you need to know. audible   WISH LIST   BEST SELLERS   NEW RELEASES Its great to be a member!  Just listen to these perks Get One Audiobook Every Month Members get 1 credit a month — good for any title regardless of length or price. Explore our catalogue Receive a BONUS audiobook each month. Editors Extra gives you a free monthly audiobook chosen by our editors as part of your membership. Add to Your Library Exclusive Member Discounts Never pay full price!  Used up your monthly credits? Members can buy additional titles at any time for 30% off the list price, plus get access to exclusive sales and daily deals. Shop Now Audible Original Podcasts Unlimited listening to gripping and immersive documentaries, comedy, and more. 100% free for members. Download to lots of devices Your library is automatically synced across all your devices, so you can listen anytime, anywhere. See listening options Need help? From app questions to listening recommendations, our Customer Care team ha

In [16]:
email_example_encoded=process_email(email_example,vocabulary)
print(email_example_encoded)

[774, 1834, 1892, 1092, 1698, 915, 1858, 960, 175, 1098, 1391, 882, 731, 1698, 161, 1024, 901, 961, 1698, 1674, 707, 574, 1061, 1024, 707, 1119, 382, 1061, 723, 665, 74, 1696, 1161, 943, 1181, 1298, 597, 1190, 1372, 196, 500, 1061, 518, 601, 711, 1892, 680, 1062, 226, 1190, 518, 115, 1211, 1161, 1894, 1025, 22, 1698, 1894, 950, 585, 1024, 459, 1097, 691, 1298, 1759, 1894, 1062, 382, 1024, 237, 23, 1696, 123, 74, 1693, 665, 1119, 1162, 1665, 960, 1298, 1264, 707, 10, 1698, 585, 1444, 73, 393, 406, 1506, 1116, 1185, 1754, 961, 1698, 73, 73, 1063, 1119, 680, 665, 1024, 486, 1698, 979, 1161, 445, 1894, 950, 876, 137, 15, 52, 1894, 445, 1537, 1892, 237, 961, 88, 1467, 961, 1180, 1092, 770, 687, 92, 1348, 1698, 961, 1375, 1190, 387, 246, 1643, 742, 725, 1892, 377, 351, 387, 246, 711, 1668, 709, 665, 952, 709, 662, 1765, 1170, 1538, 1757, 770, 12, 439, 333, 1161, 1303, 1270, 351, 387, 1492, 665, 74, 1348, 1181, 809, 1892, 1092, 119, 1892, 1894, 319, 1288, 226, 1760, 1894, 1493, 1119, 1119, 82

In [17]:
email_example_x = np.zeros(len(vocabulary))
email_example_x[email_example_encoded] = 1
email_example_x=email_example_x.reshape(1,len(vocabulary))

In [18]:
if clf.predict(email_example_x)==1:
    print('The trained model predicted: spam.')
else:
    print('The trained model predicted: not a spam.')

The trained model predicted: spam.


- train the model with a subset of the training data and see how the test accuracies compare, e.g. with 100% of training data vs with 50%.

In [24]:
from sklearn.model_selection import train_test_split

train_subset_ratio = 0.5
X_train_subset, _,  Y_train_subset, _ = train_test_split(X_train, Y_train, train_size=train_subset_ratio, random_state = 424)

clf = make_pipeline(StandardScaler(),SGDClassifier(alpha=0,learning_rate='invscaling',eta0=0.01))
clf.fit(X_train_subset, Y_train_subset)

# Training accuracy
prediction_train = clf.predict(X_train_subset)
print ('Train Accuracy:', np.mean(prediction_train == Y_train_subset) * 100,'%')
# Test accuracy
prediction_test = clf.predict(X_test)
print ('Test Accuracy:', np.mean(prediction_test == Y_test) * 100,'%')

Train Accuracy: 99.55000000000001 %
Test Accuracy: 96.0 %


In [28]:
clf = make_pipeline(StandardScaler(),SGDClassifier(alpha=0,learning_rate='invscaling',eta0=0.01))

for i in range(9):
    train_subset_ratio = (1+i)/10
    X_train_subset, _,  Y_train_subset, _ = train_test_split(X_train, Y_train, train_size=train_subset_ratio, random_state = 424)
    clf.fit(X_train_subset, Y_train_subset)
    prediction_train = clf.predict(X_train_subset)
    prediction_test = clf.predict(X_test)
    print('Train Subset Ratio: ', train_subset_ratio,' Train Accuracy:', np.mean(prediction_train == Y_train_subset) * 100,'%, ','Test Accuracy:', np.mean(prediction_test == Y_test) * 100,'%')

Train Subset Ratio:  0.1  Train Accuracy: 100.0 %,  Test Accuracy: 94.6 %
Train Subset Ratio:  0.2  Train Accuracy: 99.625 %,  Test Accuracy: 95.6 %
Train Subset Ratio:  0.3  Train Accuracy: 99.5 %,  Test Accuracy: 96.8 %
Train Subset Ratio:  0.4  Train Accuracy: 99.5 %,  Test Accuracy: 96.3 %
Train Subset Ratio:  0.5  Train Accuracy: 99.45 %,  Test Accuracy: 96.6 %
Train Subset Ratio:  0.6  Train Accuracy: 99.54166666666666 %,  Test Accuracy: 97.1 %
Train Subset Ratio:  0.7  Train Accuracy: 99.53571428571428 %,  Test Accuracy: 97.7 %
Train Subset Ratio:  0.8  Train Accuracy: 99.40625 %,  Test Accuracy: 96.89999999999999 %
Train Subset Ratio:  0.9  Train Accuracy: 99.47222222222221 %,  Test Accuracy: 97.7 %


Acknowledgements:

Adopted from problem set and course materials in Stanford CS229 and [nex3z](https://github.com/nex3z) implementation in python.